In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install wandb
!wandb

In [ ]:
%pip install kaggle --upgrade
import os
fpath = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")
if not os.path.isfile(fpath):
    os.mkdir(os.path.join(os.path.expanduser("~"), ".kaggle"))
    with open(fpath, "w") as f:
        f.write(input(f"input {fpath}"))
    !chmod 600 {fpath}

In [ ]:
if 'datasets' not in os.listdir(os.getcwd()):
    !kaggle datasets download alessiocorrado99/animals10
    !mkdir datasets
    !unzip animals10.zip -d datasets/animals10
    !rm animals10.zip

In [ ]:
import torch
import os
# torch.manual_seed(17)
from torchvision import datasets, transforms
BS = 256
IM_HEIGHT = 425
IM_WIDTH = 600
transform = transforms.Compose([transforms.Resize([IM_HEIGHT, IM_WIDTH]), transforms.ToTensor()])
path = os.path.join(os.getcwd(), "datasets", "animals10", "raw-img")
dataset = datasets.ImageFolder(path, transform=transform)
td_len, vd_len = 0.95, 0.05
train_set, val_set = torch.utils.data.random_split(dataset, [int(len(dataset)*td_len), int(len(dataset)*vd_len)+1])
train_set_loader = torch.utils.data.DataLoader(train_set, batch_size=BS, shuffle=True)
val_set_loader = torch.utils.data.DataLoader(val_set, batch_size=BS, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
i, l = next(iter(train_set_loader))
print(i.shape, i[0].shape)
sample = i[0][2]
print(sample)
plt.imshow(i[0][2].reshape(IM_HEIGHT, IM_WIDTH))
plt.show()
print(i[0][0])
print(torch.min(i[0][0]))
print(torch.max(i[0][0]))

In [ ]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
def preprocess(x, y):
    return x.to(dev), y.to(dev)
print(dev)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class someNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(IM_HEIGHT*IM_WIDTH*3, 100)
        self.l2 = nn.Linear(100, 10)
        nn.init.normal_(self.l1.weight, mean=0, std=0.1)
        nn.init.normal_(self.l2.weight, mean=0, std=0.1)
    def forward(self, x):
        x = self.l1(x)
        x = F.relu(x)
        x = self.l2(x)
        x = F.log_softmax(x)
        return x

model = someNet()
model.to(dev)

In [ ]:
WAN = True
N_EPOCH = 10
LR = 0.01

In [ ]:
if WAN:
  import wandb
  wandb.init(project="animals10", entity="0xasim")
  wandb.config = {
    "learning_rate": LR,
    "epochs": N_EPOCH,
    "batch_size": BS
  }
  run_name = wandb.name()

In [ ]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm
loss_fn = nn.CrossEntropyLoss().to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
review = lambda x_b: x_b.view(len(x_b), -1)
def getValLoss():
    with torch.no_grad():
        val_loss = torch.Tensor(len(val_set_loader)).to(dev)
        for i, (x_v_b, y_v_b) in enumerate(tqdm(val_set_loader)):
            x_v_b, y_v_b = preprocess(x_v_b, y_v_b)
            x_v_b = review(x_v_b)
            pred = model.forward(x_v_b)
            loss = loss_fn(pred, y_v_b)
            val_loss[i] = (loss.sum()/y_v_b.shape[0])
            print(val_loss)
        return val_loss.sum().item()

def train():
    for i in (range(N_EPOCH)):
        batch_loss = list()
        for bI, (x_b, y_b) in enumerate(tqdm(train_set_loader)):
            x_b = review(x_b)
            x_b, y_b = preprocess(x_b, y_b)
            pred = model.forward(x_b)
            loss = loss_fn(pred, y_b)
            # clear gradients for this training step
            optimizer.zero_grad()
            # backprop, compute gradients
            loss.backward()
            # apply gradients
            optimizer.step()
            batch_loss.append(loss.sum()/y_b.shape[0])
            if WAN:
                wandb.log({"train_loss": loss.sum().item()})
                if bI%50==0:
                    wandb.log({"val_loss": getValLoss()})

train()

In [ ]:
run_name = "true-deluge-17"
model_name = run_name+'.pt'
MODEL_PATH = os.path.join(os.getcwd(), str(N_EPOCH))
torch.save(model.state_dict(), model_name)
print(model_name)
!ls -lah {model_name}

In [ ]:
INCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
DOWNLOAD = True
if INCOLAB and DOWNLOAD:
    from google.colab import files
    files.download(model_name)